In [1]:
import numpy as np
import time
import os

# Lecture du benchmark
def read_benchmark(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    # Extraire le nombre d'éléments et de sous-ensembles
    n, m = map(int, lines[0].split())
    subsets = []
    
    for line in lines[1:]:
        subsets.append(set(map(int, line.split())))
    
    return n, m, subsets

# Fonction de fitness (maximiser le nombre d'éléments couverts)
def fitness(solution, subsets):
    covered_elements = set()
    for i, selected in enumerate(solution):
        if selected:
            covered_elements.update(subsets[i])
    return len(covered_elements)

# Initialisation des particules
def initialize_particles(num_particles, m, k):
    particles = [np.random.choice([0, 1], size=m, p=[1 - k/m, k/m]) for _ in range(num_particles)]
    velocities = [np.random.uniform(-1, 1, size=m) for _ in range(num_particles)]
    return np.array(particles), np.array(velocities)

# Fonction de mise à jour des particules (PSO discret)
def update_particle(particle, velocity, p_best, g_best, c1, c2, w, Vmax):
    inertia = w * velocity
    cognitive = c1 * np.random.uniform(0, 1, len(particle)) * (p_best - particle)
    social = c2 * np.random.uniform(0, 1, len(particle)) * (g_best - particle)
    
    velocity = np.clip(inertia + cognitive + social, -Vmax, Vmax)
    prob = 1 / (1 + np.exp(-velocity))  # Fonction sigmoïde
    new_particle = (np.random.rand(len(particle)) < prob).astype(int)
    return new_particle, velocity

# Implémentation de PSO pour MCP
def pso_mcp(file_path, num_particles=30, iterations=100, c1=1.5, c2=1.5, w=0.7, Vmax=4, k=5, early_stopping=10):
    n, m, subsets = read_benchmark(file_path)
    
    particles, velocities = initialize_particles(num_particles, m, k)
    p_best = particles.copy()
    p_best_scores = np.array([fitness(p, subsets) for p in particles])
    g_best = particles[np.argmax(p_best_scores)].copy()
    g_best_score = max(p_best_scores)
    
    no_improvement = 0
    start_time = time.time()
    
    for iteration in range(iterations):
        for i in range(num_particles):
            particles[i], velocities[i] = update_particle(particles[i], velocities[i], p_best[i], g_best, c1, c2, w, Vmax)
            score = fitness(particles[i], subsets)
            
            if score > p_best_scores[i]:
                p_best[i] = particles[i].copy()
                p_best_scores[i] = score
                
            if score > g_best_score:
                g_best = particles[i].copy()
                g_best_score = score
                no_improvement = 0
            
        no_improvement += 1
        print(f'Iteration {iteration+1}: Best Score = {g_best_score}')
        
        if no_improvement >= early_stopping:
            print("Early stopping triggered!")
            break
    
    end_time = time.time()
    print(f'Final Best Score: {g_best_score}, Execution Time: {end_time - start_time:.2f} seconds')
    return g_best, g_best_score

# Exécution du PSO sur un benchmark exemple
if __name__ == "__main__":
    benchmark_folder = "Benchmark"
    benchmark_file = os.path.join(benchmark_folder, "scp41.txt")
    
    best_solution, best_score = pso_mcp(benchmark_file, num_particles=50, iterations=200, c1=1.8, c2=1.8, w=0.6, Vmax=5, k=10)


IndexError: list index out of range